# Calibración de Cámara 📷⚙️

In [50]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os

OpenCV should be 4.8.0.76 Current version: 4.8.0


In [51]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

In [52]:
imgs_path = sorted(glob.glob("chessboard/*.jpeg"))
imgs = load_images(imgs_path)
print(len(imgs))

5


C:\Users\User\AppData\Local\Temp\ipykernel_2736\283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [53]:
size = (9, 6)

# Convertir a escala de grises
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

# Detectar esquinas
corners = [cv2.findChessboardCorners(image=img_gray, patternSize=size) for img_gray in imgs_gray]

# Guardar los ret
ret_values = [c[0] for c in corners]
print(sum(ret_values) == len(ret_values))

# Comprobación: primera coordenada detectada en la primera imagen
# print(corners[0])

True


In [54]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

corners_refined = [
    cv2.cornerSubPix(
        img_gray, cor[1], (3, 3), (-1, -1), criteria
    ) if cor[0] else []
    for img_gray, cor in zip(imgs_gray, corners_copy)
]

# Comprobación: primera esquina refinada de la primera imagen válida
print(corners_refined[0][0])

[[1405.8738 1024.0067]]


In [55]:
imgs_copy = copy.deepcopy(imgs)

drawed_corners_initial = [
    cv2.drawChessboardCorners(
        image=img,               
        patternSize=size,
        corners=cor[1],
        patternWasFound=ret
    ) if ret else img
    for img, cor, ret in zip(imgs_copy, corners, ret_values)
]

drawed_corners_refined = [
    cv2.drawChessboardCorners(
        image=img,
        patternSize=size,
        corners=corner,
        patternWasFound=ret
    ) if ret else img
    for img, corner, ret in zip(imgs_copy, corners_refined, ret_values)
]

In [56]:
def show_image(img, img_name="imagen"):
    cv2.imshow(img_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def write_image(img, output_folder, identificacion):
    os.makedirs(output_folder, exist_ok=True)   
    img_name = f"{output_folder}/corners{identificacion}.jpg"
    cv2.imwrite(filename=img_name, img=img)

output_folder = "output/initial_corners"

# Guardar y mostrar detecciones iniciales
for i, img in enumerate(drawed_corners_initial):
    show_image(img, f"initial_{i}")
    write_image(img, output_folder, f"initial_{i}")

output_folder = "../output/refined_corners"

# Guardar y mostrar detecciones refinadas
for i, img in enumerate(drawed_corners_refined):
    show_image(img, f"refined_{i}")
    write_image(img, output_folder, f"refined_{i}")

In [57]:
def get_chessboard_points(chessboard_shape, dx, dy):
    cols, rows = chessboard_shape 
    matrix = [(j * dx, i * dy, 0) for i in range(rows) for j in range(cols)]
    return np.array(matrix, np.float32)

In [58]:
points = get_chessboard_points((9, 6), 30, 30) 
print(points.shape)
chessboard_points = [points for ret in ret_values if ret]

(54, 3)


In [59]:
# Filter data and get only those with adequate detections
valid_corners = [corner for corner, ret in zip(corners_refined, ret_values) if ret]

# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

print("Number of valid images:", len(valid_corners) )
for valid in valid_corners:
    print(valid.shape)

Number of valid images: 5
(54, 1, 2)
(54, 1, 2)
(54, 1, 2)
(54, 1, 2)
(54, 1, 2)


In [60]:
img_size = (np.array(imgs_gray)[0].shape[1], np.array(imgs_gray)[0].shape[0])
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
    chessboard_points,
    valid_corners,
    img_size,
    None,
    None
)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))
rms

0.5905600529157087

In [61]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Extrinsics:\n", extrinsics)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[1.23599705e+03 0.00000000e+00 8.12289434e+02]
 [0.00000000e+00 1.24277839e+03 5.78128844e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[ 0.13903484 -0.48187775 -0.01645601  0.00532882  0.70090674]]
Extrinsics:
 [array([[-9.92015121e-01, -6.98509419e-02,  1.05008788e-01,
         2.19821246e+02],
       [ 1.23356935e-01, -7.10693074e-01,  6.92602644e-01,
         1.67620768e+02],
       [ 2.62500713e-02,  7.00025857e-01,  7.13634873e-01,
         4.65372253e+02]]), array([[-9.75880311e-01, -1.28769161e-01, -1.76284209e-01,
         7.65096567e+01],
       [ 4.77925047e-03, -8.19917118e-01,  5.72462294e-01,
         1.21207245e+02],
       [-2.18253930e-01,  5.57812175e-01,  8.00756392e-01,
         5.06547780e+02]]), array([[-9.75732259e-01, -1.29285548e-01, -1.76725225e-01,
         7.66101526e+01],
       [ 5.26329380e-03, -8.20700633e-01,  5.71334200e-01,
         1.22003663e+02],
       [-2.18903759e-01,  5.56539053e-01,  8.01464551e

In [ ]:
np.savez(
    "calibration_phone.npz",
    K=intrinsics,
    dist=dist_coeffs,
    img_size=np.array(img_size),
    rms=np.array([rms], dtype=np.float32),
)
print("Saved to calibration_phone.npz")

Saved to calibration_phone.npz


: 